# **Reviews Score.**

In [ ]:
import pandas as pd
import numpy as np
!pip install geopandas
!pip install pyproj
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from google.colab import drive           
drive.mount('/content/drive')
%cd /content/drive/My Drive/Colab Notebooks/Final PRoject

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 65.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1T98Lv-9QiRbA6FXEMM9Ee_7sJAAsnhPH/Colab Notebooks/Final PRoject


In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re
from textblob import TextBlob
from wordcloud import WordCloud
import seaborn as sns
import matplotlib.pyplot as plt
import cufflinks as cf
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
cf.go_offline()

import plotly.graph_objs as go
from plotly.subplots import make_subplots
import warnings

warnings.filterwarnings("ignore")
warnings.warn("this will not show")
#Import libraries for text analysis. 

### **Processing Reviews dataset.**

In [ ]:
reviews = pd.read_csv("reviews.csv")
print (f" Reviews dataset shape: {reviews.shape}")
reviews.head(10)
#Read Reviews datsaset.

 Reviews dataset shape: (1110024, 6)


,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2595,17857,2009-11-21,50679,Jean,Notre séjour de trois nuits.\r<br/>Nous avons ...
1,2595,19176,2009-12-05,53267,Cate,Great experience.
2,2595,19760,2009-12-10,38960,Anita,I've stayed with my friend at the Midtown Cast...
3,2595,34320,2010-04-09,71130,Kai-Uwe,"We've been staying here for about 9 nights, en..."
4,2595,46312,2010-05-25,117113,Alicia,We had a wonderful stay at Jennifer's charming...
5,2595,1238204,2012-05-07,1783688,Sergey,Hi to everyone!\r<br/>Would say our greatest c...
6,2595,1293632,2012-05-17,1870771,Loïc,"Jennifer was very friendly and helpful, and he..."
7,2595,2022498,2012-08-18,2124102,Melanie,This apartment is like a real castle old and u...
8,2595,4682989,2013-05-20,496053,Eric,Jennifer's place was in a great midtown locati...
9,2595,13193832,2014-05-21,13685934,Gerald,Jennifer is a very nice host. Everything is cl...


In [ ]:
listings = pd.read_csv("listings.csv")
print (f" Listings dataset shape: {listings.shape}")
listings.head()

 Listings dataset shape: (42931, 18)


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75356,-73.98559,Entire home/apt,150,30,49,2022-06-21,0.30,3,314,1,NaN
1,5121,BlissArtsSpace!,7356,Garon,Brooklyn,Bedford-Stuyvesant,40.68535,-73.95512,Private room,60,30,50,2019-12-02,0.30,2,365,0,NaN
2,5203,Cozy Clean Guest Room - Family Apt,7490,MaryEllen,Manhattan,Upper West Side,40.80380,-73.96751,Private room,75,2,118,2017-07-21,0.72,1,0,0,NaN
3,5178,Large Furnished Room Near B'way,8967,Shunichi,Manhattan,Midtown,40.76457,-73.98317,Private room,68,2,575,2023-02-19,3.41,1,106,52,NaN
4,5136,"Large Sunny Brooklyn Duplex, Patio + Garden",7378,Rebecca,Brooklyn,Sunset Park,40.66265,-73.99454,Entire home/apt,275,60,3,2022-08-10,0.03,1,181,1,NaN


In [ ]:
#Find common and non-common IDs.
commonIDs = set(reviews['listing_id']) & set(listings['id'])
non_commonIDs = set(reviews['listing_id']) - set(listings['id']) #All Airbnb listings in Listings has reviews in Reviews dataset.

#Count common and non-common IDs.
commonCount = len(commonIDs)
non_commonCount = len(non_commonIDs)

#Find duplicate IDs (count) of each listings. Store in dictionary.
duplicate_countsReviews = reviews['listing_id'].duplicated(keep=False).groupby(reviews['listing_id']).sum().astype(int)
listing_duplicatesReviews = duplicate_countsReviews[duplicate_countsReviews > 1].sort_values(ascending=False).to_dict()
duplicate_countsListings = listings['id'].duplicated(keep=False).groupby(reviews['id']).sum().astype(int)
listing_duplicatesListings = duplicate_countsListings[duplicate_countsListings > 1].sort_values(ascending=False).to_dict()

print(f"Common IDs: {commonIDs}")
print(f"Non-Common IDs: {non_commonIDs}")

print(f"Common IDs Count: {commonCount}")
print(f"Non-Common IDs Count: {non_commonCount}")

print(f"Listing duplicate in Reviews: {listing_duplicatesReviews}")
print(f"Listing duplicate in Listings: {listing_duplicatesListings}")

Common IDs: {48234496, 48758785, 27000837, 735051469043793926, 14155795, 15597600, 768291219355992105, 18481196, 49152044, 19791924, 52822073, 660640792226824250, 552208563699318844, 543641156240146494, 37093438, 833207052253134912, 710736060242722879, 7733315, 40501316, 718356501587296325, 588288327470809159, 11927624, 26607691, 54263886, 52166736, 53215329, 704558746717651043, 48758887, 53608551, 38797417, 4456555, 13762667, 655472, 31457396, 746276890198278272, 10748040, 15728780, 48365708, 34603173, 701639167267438757, 8388776, 16122024, 30015658, 54395062, 43778233, 4194494, 22020298, 683971575377952973, 7602400, 747245881259589857, 16122082, 42336485, 9699559, 52691176, 6422762, 39059690, 16777456, 7602426, 786685, 262405, 653534466197553419, 28180748, 12452110, 18874640, 22282513, 6553874, 735423541309079837, 7340318, 781316394662756644, 781337076750614826, 40239407, 45744441, 40239419, 822744598604546363, 654333554593235262, 798836535910859073, 53215556, 29491529, 52822346, 528

In [ ]:
#Find unique IDs in each individual data frames.
unique_idsReviews = reviews['listing_id'].unique()
unique_idsListings = listings['id'].unique()

print(f"Unique IDs in Reviews: {unique_idsReviews}")
print(f"Unique IDs in Listings: {unique_idsListings}")
print(f"Number of Unique IDs in Reviews: {len(unique_idsReviews)}")
print(f"Number of Unique IDs in Listings: {len(unique_idsListings)}")

Unique IDs in Reviews: [              2595               5121               5136 ...
 835703681287484641 832049090494847039 832368062040684030]
Unique IDs in Listings: [              2595               5121               5203 ...
 839797749155954018 839814083143454171 839817199240589949]
Number of Unique IDs in Reviews: 32627
Number of Unique IDs in Listings: 42931


In [ ]:
reviews.nunique() #Count unique values of each column in Reviews dataset.

listing_id         32627
id               1109973
date                4747
reviewer_id       953081
reviewer_name     126696
comments         1058576
dtype: int64

### **Generate scoring system.**

In [ ]:
reviewsData = reviews[["listing_id", "comments"]]
reviewsData.head()

,listing_id,comments
0,2595,Notre séjour de trois nuits.\r<br/>Nous avons ...
1,2595,Great experience.
2,2595,I've stayed with my friend at the Midtown Cast...
3,2595,"We've been staying here for about 9 nights, en..."
4,2595,We had a wonderful stay at Jennifer's charming...


In [ ]:
from nltk.stem.porter import PorterStemmer
from tqdm.notebook import tqdm
ps = PorterStemmer()
#Text normalization - stemming algorithm.

from nltk.sentiment import SentimentIntensityAnalyzer
#Import library to perform sentiment analysis.
nltk.downloader.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
#Example
sia.polarity_scores("This product is awesome")

{'neg': 0.0, 'neu': 0.423, 'pos': 0.577, 'compound': 0.6249}

In [ ]:
def stem(text):
  '''Takes input text and stem it to simple version of each word.
  Ex: 'running' --> 'run' '''
    ps = PorterStemmer()
    y = []
    if isinstance(text, str):  #Check if text is a non-null string.
        for i in text.split():
            y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
reviewsData["comments"] = tqdm(reviewsData["comments"].apply(lambda x: ps.stem(x) if isinstance(x, str) else x))
#Apply def stem to each "comments" instance in reviews dataset.

  0%|          | 0/1110024 [00:00<?, ?it/s]

### **Run polarity score on all reviews. Performs sentiment analysis directly on pre-processed Reviews.**

In [ ]:
orig_resPolarity = {}

#Apply sentiment analysis to the "comments" column with tqdm progress bar.
for index, row in tqdm(reviewsData.iterrows(), total=reviewsData.shape[0]):
    if isinstance(row["comments"], str):
        text = row["comments"]
        id = row["listing_id"]
        orig_resPolarity[id] = sia.polarity_scores(text)

  0%|          | 0/1110024 [00:00<?, ?it/s]

In [ ]:
orig_resPolarity=pd.DataFrame(orig_resPolarity).T
print(orig_resPolarity.shape)
orig_resPolarity=orig_resPolarity.reset_index().rename(columns={'index':"id"})
orig_resPolarity.head()

(32625, 4)


,id,neg,neu,pos,compound
0,2595,0.035,0.810,0.155,0.9883
1,5121,0.000,0.546,0.454,0.9668
2,5136,0.000,0.607,0.393,0.9682
3,5586,0.000,0.617,0.383,0.9659
4,5178,0.557,0.114,0.330,-0.2500


In [ ]:
orig_resPolarity.groupby('id')['compound'].agg(list)
#Group the sentiment analysis results (original text) stored in the dictionary by the "id" and aggregate the "compound" scores into a list for each group.

id
2595                  [0.9883]
5121                  [0.9668]
5136                  [0.9682]
5178                   [-0.25]
5203                  [0.9903]
                        ...   
835703681287484641    [0.8393]
837075395576130377    [0.9365]
838131275214287871    [0.9738]
838548659003355204    [0.9781]
839119919829213270    [0.9701]
Name: compound, Length: 32625, dtype: object

## **RoBERTa pretrained model.**

In [ ]:
!pip install transformers
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
#Import transformer library and modules for sentiment analysis.

from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')  
nltk.download('punkt')
#Import WordNetLemmatizer to sort words. 
#WordNet corpus - a lexical database to lemmatize each word into dictionary form (nouns, verbs, adjs, adverbs). 
#Punkt tokenizer - a pre-trained tokenizer used for tokenizing text into sentences or words. 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 96.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.6 MB/s eta 0:00:00


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
def lemmatizeText(text):
  '''Lemmatize input text into dictionary based words.'''
  lemmatizer = WordNetLemmatizer()
  lemmatizedTokens = [lemmatizer.lemmatize(token) for token in nltk.word_tokenize(text)]
  lemmatizedText = ' '.join(lemmatizedTokens)
  return lemmatizedText

In [ ]:
lemmatizedComments = []

for comment in tqdm(reviewsData["comments"]):
    if isinstance(comment, str):
        lemmatizedComment = lemmatizeText(comment)
        lemmatizedComments.append(lemmatizedComment)
    else:
        lemmatizedComments.append(None)  
        #Store None for non-string values.

reviewsData["lemmatizedComments"] = lemmatizedComments
#Lemmatize each 'comments' in Reviews dataset. 

  0%|          | 0/1110024 [00:00<?, ?it/s]

In [ ]:
reviewsData.head()
#ReviewsData should have an additional column for lemmatized reviews. 

,listing_id,comments,lemmatizedComments
0,2595,notre séjour de trois nuits.\r<br/>nous avons ...,notre séjour de trois nuits . < br/ > nous avo...
1,2595,great experience.,great experience .
2,2595,i've stayed with my friend at the midtown cast...,i 've stayed with my friend at the midtown cas...
3,2595,"we've been staying here for about 9 nights, en...","we 've been staying here for about 9 night , e..."
4,2595,we had a wonderful stay at jennifer's charming...,we had a wonderful stay at jennifer 's charmin...


### **Run polarity score on all reviews. Performs sentiment analysis on the pre-processed text Reviews trained by RoBERTa algorithm.**

In [ ]:
def polarity_scoreRoberta(example):
  '''Take input text, tokenize using RoBERTa and returns Polarity scores as dictionary form.'''

  encodedText = tokenizer (example, return_tensors='pt')
  #Tokenize the input text using RoBERTa tokenizer and encode it as PyTorch framework format tensors.
  output = model(**encodedText)
  #Pass encodedText to pre-trained RoBERTa model to perform sequence classification.
  scores = output[0][0].detach().numpy()
  scores=softmax(scores)
  #Retrieve the scores from the model's output and apply softmax normalization to ensure scores are transformed into a probability distribution.
  scoresDict= {"roberta_neg": scores[0],
              "roberta_neu": scores[1],
              "roberta_pos": scores[2]}
  return scoresDict
  #Return Polarity score as dictionary form for each 'comments'.

**roberta_resPolarity result took 21 hours.**

In [ ]:
roberta_resPolarity = []

#Apply sentiment analysis to the "comments" column with tqdm progress bar.
for i, row in tqdm(reviewsData.iterrows(), total=len(reviewsData)):
    try:
      text = row["comments"]
      idValue = row["listing_id"]
      roberta_scorePolarity = polarity_scoreRoberta(text)
      #Obtain polarity score from RoBERTa pre-trained model.
      roberta_resPolarity.append((idValue, roberta_scorePolarity))
    except:
      print(f"Listing ID {idValue} caused error")

  0%|          | 0/1110024 [00:00<?, ?it/s]

Listing ID 2595 caused error
Listing ID 2595 caused error
Listing ID 2595 caused error
Listing ID 2595 caused error


### **Write Polarity scores from RoBERTa model to a csv file.**

In [ ]:
import csv
filename = "reviewsScore.csv"
header = ["id_value", "polarity_score"]

with open(filename, "a", newline="") as csvfile:
    writer = csv.writer(csvfile)
    #Check if the file is empty and write the header if needed.
    if csvfile.tell() == 0:
        writer.writerow(header)
    writer.writerows(roberta_resPolarity)

print(f"Data appended to {filename} successfully.")

In [ ]:
reviewsScore = pd.read_csv("reviewsScore.xls")
print (f" Reviews score dataset shape: {reviewsScore.shape}")
uniqueCount = reviewsScore["id_value"].nunique()
print(uniqueCount)
#Read Reviews datsaset.

 Reviews score dataset shape: (1106421, 2)
32607


In [ ]:
reviewsScore.head(10)

,id_value,polarity_score
0,2595,"{'roberta_neg': 0.13033417, 'roberta_neu': 0.8..."
1,2595,"{'roberta_neg': 0.0071270172, 'roberta_neu': 0..."
2,2595,"{'roberta_neg': 0.0017230441, 'roberta_neu': 0..."
3,2595,"{'roberta_neg': 0.0014082947, 'roberta_neu': 0..."
4,2595,"{'roberta_neg': 0.0012992539, 'roberta_neu': 0..."
5,2595,"{'roberta_neg': 0.0011575663, 'roberta_neu': 0..."
6,2595,"{'roberta_neg': 0.0016218657, 'roberta_neu': 0..."
7,2595,"{'roberta_neg': 0.0829413, 'roberta_neu': 0.31..."
8,2595,"{'roberta_neg': 0.001730755, 'roberta_neu': 0...."
9,2595,"{'roberta_neg': 0.0028763283, 'roberta_neu': 0..."
